# Binning a ground schedule

In this notebook, we take an observing schedule from `toast_ground_sim.py` and translate it into a depth map.

In [ ]:
# Capture C++ output in the jupyter cells
%reload_ext wurlitzer

First, we need a focalplane. If one does not already exist, TOAST `pipelines` includes a tool for generating mock hexagonal focalplanes:

In [ ]:
! toast_fake_focalplane.py --help

Here we create a focalplane with 10-degree FOV and a mininimum of 20 pixels:

In [ ]:
! toast_fake_focalplane.py \
    --minpix 20 \
    --out focalplane \
    --fwhm 30 \
    --fov 10 \
    --psd_fknee 5e-2 \
    --psd_NET 1e-3 \
    --psd_alpha 1 \
    --psd_fmin 1e-5

The actual focalplane ends up having 37 pixels, instead of the minimum of 20.  This is because regular packing of the hexagon is quantized.  Notice that the final name of the focalplane is `focalplane_37.pkl`.  We'll need the name to run the simulation script.  We also need a schedule file.  This may already exist if you previously ran the `simscan_ground` notebook, but we'll re-create it here just in case:

In [ ]:
! toast_ground_schedule.py \
    --site-lat "-22.958064" \
    --site-lon "-67.786222" \
    --site-alt 5200 \
    --site-name Atacama \
    --telescope LAT \
    --start "2020-01-01 00:00:00" \
    --stop "2020-01-01 12:00:00" \
    --patch-coord C \
    --patch small_patch,1,40,-40,44,-44 \
    --out schedule.txt

We will use the versatile ground simulation pipeline, `toast_ground_sim.py`, to bin the map.  It will be covered in detail in lesson 7 so here we simply write out a parameter file:

In [ ]:
%%writefile bin_schedule.par
--sample-rate
10.0
--scan-rate
0.3
--scan-accel
10.0
--nside
64
--focalplane
focalplane_37.pkl
--schedule
schedule.txt
--out
out
--simulate-noise
--freq
100
--no-destripe
--no-binmap
--hits
--wcov

Now we run the pipeline with multiple MPI processes that divide into processing groups:

In [ ]:
! toast_ground_sim.py @bin_schedule.par

Let's examine the resulting hits and depth map

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import healpy

hits = healpy.read_map("out/00000000/100/toast_100_telescope_all_time_all_hmap.fits")
hits[hits == 0] = healpy.UNSEEN
healpy.mollview(hits, unit="hits", title="Total hits")
healpy.graticule(22.5, verbose=False)

In [ ]:
wcov = healpy.read_map("out/00000000/100/toast_100_telescope_all_time_all_wcov.fits")
wcov *= 1e12  # from K^2 to uK^2
wcov[wcov == 0] = healpy.UNSEEN
healpy.mollview(wcov, unit="$\mu$K$^2$", title="White noise variance", min=1e0, max=1e3)
healpy.graticule(22.5, verbose=False)